# Medical OCR Pipeline - Docker/MCP Service Launcher & Pipeline Demo

This notebook demonstrates how to launch all Docker/MCP services in parallel xterm windows and validate the complete medical OCR pipeline step-by-step.

## Features:
- **Parallel Service Launch**: Start 11 OCR engines in separate xterm windows
- **Go Integration**: Interface with Go-based service components  
- **Real-time Monitoring**: Track service health and pipeline status
- **Step-by-step Validation**: Test each pipeline stage systematically
- **Comprehensive Testing**: Validate all 11 OCR engines with sample documents

## OCR Engines (11 Total):
1. **Traditional**: Tesseract (8089), EasyOCR (8092), PaddleOCR (8090)
2. **Modern**: Surya (8091), Docling (8093), DocTR (8094)  
3. **Vision-Language**: DeepSeek-VL (8095), Qwen3-VL (8096)
4. **Specialized**: Marker (8097), Nanonets (8098), Chandra (8099)

## 1. Import Required Libraries

Import necessary libraries for subprocess management, Docker SDK, terminal handling, and concurrent execution.

In [ ]:
import subprocess
import threading
import time
import json
import os
import signal
import sys
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any, Optional

# HTTP and Docker libraries
import requests
try:
    import docker
    DOCKER_AVAILABLE = True
except ImportError:
    print("WARNING: Docker SDK not available. Install with: pip install docker")
    DOCKER_AVAILABLE = False

# Data analysis libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Display configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('default')

print("All libraries imported successfully")
print(f"Docker SDK available: {DOCKER_AVAILABLE}")
print(f"Python version: {sys.version.split()[0]}")

## 2. Docker Service Management Functions

Create functions to start, stop, and manage Docker containers and services programmatically.

In [ ]:
class DockerServiceManager:
    """Manage Docker services for the medical OCR pipeline"""
    
    def __init__(self, project_root: str = ".."):
        self.project_root = Path(project_root).absolute()
        self.docker_client = None
        self.compose_file = self.project_root / "docker-compose.yml"
        
        if DOCKER_AVAILABLE:
            try:
                self.docker_client = docker.from_env()
                print("✅ Docker client connected")
            except Exception as e:
                print(f"⚠️ Docker client connection failed: {e}")
    
    def check_docker_compose(self) -> bool:
        """Check if docker-compose is available and config is valid"""
        try:
            result = subprocess.run(
                ["docker-compose", "config"], 
                cwd=self.project_root,
                capture_output=True, 
                text=True,
                timeout=30
            )
            if result.returncode == 0:
                print("✅ docker-compose.yml is valid")
                return True
            else:
                print(f"❌ docker-compose config error: {result.stderr}")
                return False
        except subprocess.TimeoutExpired:
            print("⏰ docker-compose config check timed out")
            return False
        except FileNotFoundError:
            print("❌ docker-compose command not found")
            return False
        except Exception as e:
            print(f"💥 docker-compose check failed: {e}")
            return False
    
    def build_all_images(self) -> bool:
        """Build all Docker images for OCR services"""
        try:
            print("🔨 Building all Docker images...")
            result = subprocess.run(
                ["./scripts/build_docker_images.sh"],
                cwd=self.project_root,
                capture_output=True,
                text=True,
                timeout=1800  # 30 minutes timeout
            )
            
            if result.returncode == 0:
                print("✅ All Docker images built successfully")
                return True
            else:
                print(f"❌ Build failed: {result.stderr}")
                return False
                
        except subprocess.TimeoutExpired:
            print("⏰ Build process timed out (30 minutes)")
            return False
        except Exception as e:
            print(f"💥 Build failed: {e}")
            return False
    
    def start_all_services(self, detached: bool = True) -> bool:
        """Start all services using docker-compose"""
        try:
            cmd = ["docker-compose", "up"]
            if detached:
                cmd.append("-d")
            
            print(f"🚀 Starting all services (detached={detached})...")
            result = subprocess.run(
                cmd,
                cwd=self.project_root,
                capture_output=True,
                text=True,
                timeout=300  # 5 minutes timeout
            )
            
            if result.returncode == 0:
                print("✅ All services started successfully")
                if detached:
                    time.sleep(10)  # Give services time to initialize
                return True
            else:
                print(f"❌ Service start failed: {result.stderr}")
                return False
                
        except subprocess.TimeoutExpired:
            print("⏰ Service start timed out")
            return False
        except Exception as e:
            print(f"💥 Service start failed: {e}")
            return False
    
    def stop_all_services(self) -> bool:
        """Stop all services"""
        try:
            print("🛑 Stopping all services...")
            result = subprocess.run(
                ["docker-compose", "down"],
                cwd=self.project_root,
                capture_output=True,
                text=True,
                timeout=120
            )
            
            if result.returncode == 0:
                print("✅ All services stopped successfully")
                return True
            else:
                print(f"❌ Service stop failed: {result.stderr}")
                return False
                
        except Exception as e:
            print(f"💥 Service stop failed: {e}")
            return False
    
    def get_service_status(self) -> Dict[str, Any]:
        """Get status of all services"""
        try:
            result = subprocess.run(
                ["docker-compose", "ps", "--format", "json"],
                cwd=self.project_root,
                capture_output=True,
                text=True,
                timeout=30
            )
            
            if result.returncode == 0:
                services = []
                for line in result.stdout.strip().split('\n'):
                    if line.strip():
                        try:
                            services.append(json.loads(line))
                        except json.JSONDecodeError:
                            continue
                return {"status": "success", "services": services}
            else:
                return {"status": "error", "message": result.stderr}
                
        except Exception as e:
            return {"status": "error", "message": str(e)}

# Initialize Docker service manager
docker_manager = DockerServiceManager()

# Test Docker setup
print("🔍 Testing Docker setup...")
print(f"📁 Project root: {docker_manager.project_root}")
print(f"🐳 Docker compose file: {docker_manager.compose_file}")
print(f"📋 Compose valid: {docker_manager.check_docker_compose()}")

## 3. Xterm Window Management

Implement functions to open and manage multiple xterm terminal windows for service monitoring.

In [ ]:
class XtermWindowManager:
    """Manage xterm windows for service monitoring"""
    
    def __init__(self):
        self.windows = {}  # Track opened windows
        self.processes = {}  # Track window processes
    
    def check_xterm_available(self) -> bool:
        """Check if xterm is available on the system"""
        try:
            result = subprocess.run(
                ["which", "xterm"], 
                capture_output=True, 
                text=True, 
                timeout=10
            )
            available = result.returncode == 0
            
            if available:
                print("✅ xterm is available")
            else:
                print("❌ xterm not found - install with: brew install xterm (macOS) or apt install xterm (Linux)")
            
            return available
        except Exception as e:
            print(f"💥 xterm check failed: {e}")
            return False
    
    def open_service_monitor(self, service_name: str, port: int, docker_service: str) -> bool:
        """Open xterm window to monitor a specific service"""
        try:
            window_title = f"MCP OCR - {service_name.upper()} (Port {port})"
            geometry = f"100x30+{200 + (port - 8089) * 150}+{100 + (port - 8089) * 50}"
            
            # Command to run in xterm
            monitor_cmd = f"""
            echo '🚀 Starting {service_name} monitoring...';
            echo '📡 Service: {docker_service}';
            echo '🔌 Port: {port}';
            echo '═══════════════════════════════════';
            echo '';
            docker-compose logs -f {docker_service};
            echo '';
            echo '💀 Service monitoring stopped.';
            echo 'Press Enter to close window...';
            read
            """
            
            # Start xterm with monitoring
            process = subprocess.Popen([
                "xterm",
                "-T", window_title,
                "-geometry", geometry,
                "-e", f"bash -c '{monitor_cmd}'"
            ], cwd=docker_manager.project_root)
            
            self.windows[service_name] = {
                "port": port,
                "docker_service": docker_service,
                "title": window_title,
                "geometry": geometry
            }
            self.processes[service_name] = process
            
            print(f"✅ Opened monitoring window for {service_name} (PID: {process.pid})")
            return True
            
        except Exception as e:
            print(f"❌ Failed to open window for {service_name}: {e}")
            return False
    
    def open_health_monitor(self) -> bool:
        """Open dedicated health monitoring window"""
        try:
            window_title = "OCR Pipeline - Health Monitor"
            geometry = "120x40+50+50"
            
            monitor_cmd = f"""
            echo '🔍 OCR Pipeline Health Monitor';
            echo '═══════════════════════════════';
            echo '';
            cd '{docker_manager.project_root}';
            while true; do
                echo "🕐 $(date)";
                echo '─────────────────────────────────';
                python scripts/health_check.py;
                echo '';
                echo 'Next check in 30 seconds...';
                echo '';
                sleep 30;
            done
            """
            
            process = subprocess.Popen([
                "xterm",
                "-T", window_title,
                "-geometry", geometry,
                "-e", f"bash -c '{monitor_cmd}'"
            ])
            
            self.windows["health_monitor"] = {
                "title": window_title,
                "geometry": geometry
            }
            self.processes["health_monitor"] = process
            
            print(f"✅ Opened health monitoring window (PID: {process.pid})")
            return True
            
        except Exception as e:
            print(f"❌ Failed to open health monitor: {e}")
            return False
    
    def open_docker_compose_monitor(self) -> bool:
        """Open Docker Compose central monitoring window"""
        try:
            window_title = "Docker Compose - All Services"
            geometry = "140x50+800+50"
            
            monitor_cmd = f"""
            echo '🐳 Docker Compose - All Services Monitor';
            echo '═══════════════════════════════════════';
            echo '';
            cd '{docker_manager.project_root}';
            echo '📊 Service Status:';
            docker-compose ps;
            echo '';
            echo '📋 Following all service logs...';
            echo '═══════════════════════════════════════';
            docker-compose logs -f;
            echo '';
            echo '💀 All services stopped.';
            echo 'Press Enter to close...';
            read
            """
            
            process = subprocess.Popen([
                "xterm",
                "-T", window_title,
                "-geometry", geometry,
                "-e", f"bash -c '{monitor_cmd}'"
            ])
            
            self.windows["docker_compose"] = {
                "title": window_title,
                "geometry": geometry
            }
            self.processes["docker_compose"] = process
            
            print(f"✅ Opened Docker Compose monitor (PID: {process.pid})")
            return True
            
        except Exception as e:
            print(f"❌ Failed to open Docker Compose monitor: {e}")
            return False
    
    def close_all_windows(self):
        """Close all opened xterm windows"""
        closed_count = 0
        for name, process in self.processes.items():
            try:
                process.terminate()
                process.wait(timeout=5)
                closed_count += 1
                print(f"✅ Closed window: {name}")
            except subprocess.TimeoutExpired:
                process.kill()
                print(f"⚠️ Force killed window: {name}")
                closed_count += 1
            except Exception as e:
                print(f"❌ Failed to close window {name}: {e}")
        
        self.windows.clear()
        self.processes.clear()
        print(f"📊 Total windows closed: {closed_count}")
    
    def list_windows(self):
        """List all opened monitoring windows"""
        if not self.windows:
            print("📋 No monitoring windows currently open")
            return
        
        print("📋 Open Monitoring Windows:")
        print("═══════════════════════════")
        for name, info in self.windows.items():
            process = self.processes.get(name)
            status = "Running" if process and process.poll() is None else "Stopped"
            
            if name == "health_monitor":
                print(f"   🔍 {info['title']} - Status: {status}")
            elif name == "docker_compose":
                print(f"   🐳 {info['title']} - Status: {status}")
            else:
                port = info.get('port', 'N/A')
                service = info.get('docker_service', 'N/A')
                print(f"   🔧 {name.upper()} (Port {port}) - {service} - Status: {status}")

# Initialize xterm manager
xterm_manager = XtermWindowManager()

# Test xterm availability
print("🔍 Testing xterm setup...")
xterm_available = xterm_manager.check_xterm_available()
print(f"🪟 Xterm available: {xterm_available}")

## 4. Go Service Launcher Implementation

Create Go application launchers that can start MCP services and other Go-based components.

In [ ]:
class GoServiceLauncher:
    """Launch and manage Go-based services and applications"""
    
    def __init__(self):
        self.go_processes = {}
        self.go_available = self.check_go_available()
    
    def check_go_available(self) -> bool:
        """Check if Go is installed and available"""
        try:
            result = subprocess.run(
                ["go", "version"], 
                capture_output=True, 
                text=True, 
                timeout=10
            )
            
            if result.returncode == 0:
                version = result.stdout.strip()
                print(f"✅ Go is available: {version}")
                return True
            else:
                print("❌ Go not found - install from: https://golang.org/")
                return False
                
        except Exception as e:
            print(f"💥 Go check failed: {e}")
            return False
    
    def create_go_launcher_script(self) -> str:
        """Create a Go script to launch multiple services"""
        go_script = '''
package main

import (
    "context"
    "fmt"
    "log"
    "os"
    "os/exec"
    "os/signal"
    "sync"
    "syscall"
    "time"
)

type Service struct {
    Name    string
    Command string
    Args    []string
    Port    int
}

type ServiceManager struct {
    services []Service
    ctx      context.Context
    cancel   context.CancelFunc
    wg       sync.WaitGroup
}

func NewServiceManager() *ServiceManager {
    ctx, cancel := context.WithCancel(context.Background())
    return &ServiceManager{
        ctx:    ctx,
        cancel: cancel,
    }
}

func (sm *ServiceManager) AddService(name, command string, args []string, port int) {
    sm.services = append(sm.services, Service{
        Name:    name,
        Command: command,
        Args:    args,
        Port:    port,
    })
}

func (sm *ServiceManager) StartService(service Service) {
    defer sm.wg.Done()
    
    fmt.Printf("🚀 Starting %s on port %d\\n", service.Name, service.Port)
    
    cmd := exec.CommandContext(sm.ctx, service.Command, service.Args...)
    cmd.Stdout = os.Stdout
    cmd.Stderr = os.Stderr
    
    if err := cmd.Start(); err != nil {
        fmt.Printf("❌ Failed to start %s: %v\\n", service.Name, err)
        return
    }
    
    fmt.Printf("✅ %s started (PID: %d)\\n", service.Name, cmd.Process.Pid)
    
    // Wait for context cancellation or process completion
    done := make(chan error, 1)
    go func() {
        done <- cmd.Wait()
    }()
    
    select {
    case <-sm.ctx.Done():
        fmt.Printf("🛑 Stopping %s...\\n", service.Name)
        if err := cmd.Process.Signal(syscall.SIGTERM); err != nil {
            fmt.Printf("⚠️ Failed to send SIGTERM to %s: %v\\n", service.Name, err)
            cmd.Process.Kill()
        }
        <-done
    case err := <-done:
        if err != nil {
            fmt.Printf("💥 %s exited with error: %v\\n", service.Name, err)
        } else {
            fmt.Printf("✅ %s exited cleanly\\n", service.Name)
        }
    }
}

func (sm *ServiceManager) StartAll() {
    // Setup signal handling
    c := make(chan os.Signal, 1)
    signal.Notify(c, os.Interrupt, syscall.SIGTERM)
    
    // Start all services
    for _, service := range sm.services {
        sm.wg.Add(1)
        go sm.StartService(service)
        time.Sleep(100 * time.Millisecond) // Brief delay between starts
    }
    
    fmt.Printf("🎉 All %d services started\\n", len(sm.services))
    
    // Wait for signal
    <-c
    fmt.Println("\\n🛑 Shutdown signal received")
    
    // Cancel context to stop all services
    sm.cancel()
    
    // Wait for all services to stop
    sm.wg.Wait()
    fmt.Println("✅ All services stopped")
}

func main() {
    sm := NewServiceManager()
    
    // Add OCR services (these would normally be Docker services)
    // For demonstration, we'll add some simple services
    sm.AddService("health-check", "python", []string{"scripts/health_check.py", "--wait", "5"}, 0)
    
    fmt.Println("🔧 Go Service Manager - Medical OCR Pipeline")
    fmt.Println("═══════════════════════════════════════════")
    
    sm.StartAll()
}
'''
        
        # Write Go script to file
        go_file = docker_manager.project_root / "scripts" / "go_service_launcher.go"
        go_file.write_text(go_script)
        
        print(f"📝 Created Go launcher script: {go_file}")
        return str(go_file)
    
    def build_go_launcher(self, go_file: str) -> Optional[str]:
        """Build the Go launcher application"""
        if not self.go_available:
            print("❌ Go not available, cannot build launcher")
            return None
        
        try:
            binary_path = docker_manager.project_root / "scripts" / "go_service_launcher"
            
            print("🔨 Building Go service launcher...")
            result = subprocess.run(
                ["go", "build", "-o", str(binary_path), go_file],
                cwd=docker_manager.project_root,
                capture_output=True,
                text=True,
                timeout=60
            )
            
            if result.returncode == 0:
                print(f"✅ Go launcher built: {binary_path}")
                return str(binary_path)
            else:
                print(f"❌ Go build failed: {result.stderr}")
                return None
                
        except Exception as e:
            print(f"💥 Go build failed: {e}")
            return None
    
    def run_go_launcher(self, binary_path: str, detached: bool = True) -> Optional[subprocess.Popen]:
        """Run the Go service launcher"""
        try:
            if detached:
                print("🚀 Starting Go service launcher (detached)...")
                process = subprocess.Popen(
                    [binary_path],
                    cwd=docker_manager.project_root,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE
                )
                
                self.go_processes["launcher"] = process
                print(f"✅ Go launcher started (PID: {process.pid})")
                return process
            else:
                print("🚀 Starting Go service launcher (foreground)...")
                result = subprocess.run(
                    [binary_path],
                    cwd=docker_manager.project_root,
                    timeout=300
                )
                
                if result.returncode == 0:
                    print("✅ Go launcher completed successfully")
                else:
                    print(f"❌ Go launcher failed with code: {result.returncode}")
                
                return None
                
        except Exception as e:
            print(f"💥 Go launcher execution failed: {e}")
            return None
    
    def stop_go_processes(self):
        """Stop all Go processes"""
        stopped_count = 0
        for name, process in self.go_processes.items():
            try:
                process.terminate()
                process.wait(timeout=10)
                stopped_count += 1
                print(f"✅ Stopped Go process: {name}")
            except subprocess.TimeoutExpired:
                process.kill()
                print(f"⚠️ Force killed Go process: {name}")
                stopped_count += 1
            except Exception as e:
                print(f"❌ Failed to stop Go process {name}: {e}")
        
        self.go_processes.clear()
        print(f"📊 Total Go processes stopped: {stopped_count}")

# Initialize Go service launcher
go_launcher = GoServiceLauncher()

# Create and build Go launcher if Go is available
if go_launcher.go_available:
    go_script_path = go_launcher.create_go_launcher_script()
    go_binary_path = go_launcher.build_go_launcher(go_script_path)
    print(f"🔧 Go launcher ready: {go_binary_path is not None}")
else:
    print("⚠️ Go launcher not available (Go not installed)")
    go_binary_path = None

## 5. Parallel Process Execution

Implement concurrent execution of multiple services using threading and multiprocessing.

In [ ]:
# OCR Services Configuration
OCR_SERVICES = {
    'tesseract': {'port': 8089, 'docker_service': 'mcp-tesseract'},
    'easyocr': {'port': 8092, 'docker_service': 'mcp-easyocr'},
    'paddle': {'port': 8090, 'docker_service': 'mcp-paddle'},
    'surya': {'port': 8091, 'docker_service': 'mcp-surya'},
    'docling': {'port': 8093, 'docker_service': 'mcp-docling'},
    'doctr': {'port': 8094, 'docker_service': 'mcp-doctr'},
    'deepseek': {'port': 8095, 'docker_service': 'mcp-deepseek'},
    'qwen': {'port': 8096, 'docker_service': 'mcp-qwen'},
    'marker': {'port': 8097, 'docker_service': 'mcp-marker'},
    'nanonets': {'port': 8098, 'docker_service': 'mcp-nanonets'},
    'chandra': {'port': 8099, 'docker_service': 'mcp-chandra'}
}

class ParallelServiceManager:
    """Manage parallel execution of services and monitoring"""
    
    def __init__(self):
        self.executor = ThreadPoolExecutor(max_workers=15)
        self.running_tasks = {}
        self.service_status = {}
    
    def start_all_services_parallel(self, mode: str = "docker_compose") -> Dict[str, Any]:
        """Start all services in parallel with different modes"""
        results = {"mode": mode, "services": {}, "monitoring": {}}
        
        if mode == "docker_compose":
            return self._start_docker_compose_mode()
        elif mode == "individual_xterm":
            return self._start_individual_xterm_mode()
        elif mode == "background_only":
            return self._start_background_only_mode()
        else:
            raise ValueError(f"Unknown mode: {mode}")
    
    def _start_docker_compose_mode(self) -> Dict[str, Any]:
        """Start all services via docker-compose with monitoring windows"""
        print("🐳 Starting all services via Docker Compose...")
        
        # Start services
        docker_success = docker_manager.start_all_services(detached=True)
        if not docker_success:
            return {"success": False, "error": "Failed to start Docker services"}
        
        # Start monitoring windows
        monitoring_results = {}
        
        if xterm_available:
            # Health monitor
            health_task = self.executor.submit(xterm_manager.open_health_monitor)
            monitoring_results["health_monitor"] = health_task
            
            # Docker compose monitor  
            compose_task = self.executor.submit(xterm_manager.open_docker_compose_monitor)
            monitoring_results["docker_compose"] = compose_task
            
            # Wait briefly for windows to open
            time.sleep(3)
        
        return {
            "success": True,
            "mode": "docker_compose",
            "services": OCR_SERVICES,
            "monitoring": monitoring_results
        }
    
    def _start_individual_xterm_mode(self) -> Dict[str, Any]:
        """Start each service in its own xterm window"""
        print("🔧 Starting services in individual xterm windows...")
        
        if not xterm_available:
            return {"success": False, "error": "xterm not available"}
        
        # Start each service in parallel
        service_tasks = {}
        for service_name, config in OCR_SERVICES.items():
            task = self.executor.submit(
                xterm_manager.open_service_monitor,
                service_name,
                config['port'],
                config['docker_service']
            )
            service_tasks[service_name] = task
            time.sleep(1)  # Brief delay between window opens
        
        # Start Docker services in background
        docker_task = self.executor.submit(docker_manager.start_all_services, True)
        
        return {
            "success": True,
            "mode": "individual_xterm",
            "services": service_tasks,
            "docker_task": docker_task
        }
    
    def _start_background_only_mode(self) -> Dict[str, Any]:
        """Start services in background without monitoring windows"""
        print("🔧 Starting services in background only...")
        
        docker_success = docker_manager.start_all_services(detached=True)
        
        return {
            "success": docker_success,
            "mode": "background_only",
            "services": OCR_SERVICES
        }
    
    def run_parallel_health_checks(self) -> Dict[str, Any]:
        """Run health checks for all services in parallel"""
        print("🔍 Running parallel health checks...")
        
        def check_single_service(service_name: str, port: int):
            url = f"http://localhost:{port}/health"
            try:
                start_time = time.time()
                response = requests.get(url, timeout=5)
                response_time = time.time() - start_time
                
                if response.status_code == 200:
                    return {
                        "service": service_name,
                        "status": "healthy",
                        "response_time": response_time,
                        "data": response.json()
                    }
                else:
                    return {
                        "service": service_name,
                        "status": "unhealthy",
                        "response_time": response_time,
                        "error": f"HTTP {response.status_code}"
                    }
            except Exception as e:
                return {
                    "service": service_name,
                    "status": "error",
                    "error": str(e)
                }
        
        # Submit all health checks in parallel
        health_tasks = {}
        for service_name, config in OCR_SERVICES.items():
            task = self.executor.submit(check_single_service, service_name, config['port'])
            health_tasks[service_name] = task
        
        # Collect results
        health_results = {}
        for service_name, task in health_tasks.items():
            try:
                result = task.result(timeout=10)
                health_results[service_name] = result
            except Exception as e:
                health_results[service_name] = {
                    "service": service_name,
                    "status": "timeout",
                    "error": str(e)
                }
        
        return health_results
    
    def stop_all_parallel(self):
        """Stop all services and monitoring in parallel"""
        print("🛑 Stopping all services and monitoring...")
        
        # Stop monitoring windows
        if xterm_available:
            stop_xterm_task = self.executor.submit(xterm_manager.close_all_windows)
        
        # Stop Go processes
        stop_go_task = self.executor.submit(go_launcher.stop_go_processes)
        
        # Stop Docker services
        stop_docker_task = self.executor.submit(docker_manager.stop_all_services)
        
        # Wait for all to complete
        tasks = [stop_docker_task, stop_go_task]
        if xterm_available:
            tasks.append(stop_xterm_task)
        
        for task in tasks:
            try:
                task.result(timeout=30)
            except Exception as e:
                print(f"⚠️ Stop task failed: {e}")
        
        print("✅ All services stopped")
    
    def shutdown(self):
        """Shutdown the executor"""
        self.executor.shutdown(wait=True)

# Initialize parallel service manager
parallel_manager = ParallelServiceManager()

print("🔧 Parallel Service Manager initialized")
print(f"📊 Available services: {len(OCR_SERVICES)}")
print(f"🔄 Max workers: {parallel_manager.executor._max_workers}")